# Scikit learn

Recuerden que tiene una muy buena [documentación](https://scikit-learn.org/stable/user_guide.html).

Vimos que en general, los objetos de sklearn tienen los métodos:

- fit
- transform o predict

Vamos a trabajar con un dataset sacado de una competencia de data science para aprender a utilizar skitlearn.

Montamos drive

In [ ]:
#from google.colab import drive # La usamos para montar nuestra unidad de Google Drive
#drive.mount('/content/drive') # Montamos nuestra unidad de Google Drive

Mounted at /content/drive


Vamos a trabajar con un dataset que contiene datos sobre la actividad de los usuarios que visitan un sitio web de ecommerce.

Las columnas del dataset son:

- id: Id del usuario
- administrative: Número de veces que el usuario visito la sección "administrative"
- administrative_duration: Tiempo que el usuario paso en la sección administrative
- informational: Número de veces que el usuario visitó la sección "informational"
- informational_duration: Tiempo que el usuario paso en la sección informational
- productrelated: Número de veces que el usuario visitó la sección "products related"
- productrelated_duration: Tiempo que el usuario pasó en la sección
- bouncerates: Porcentaje de visitantes que entran a la página e inmediatamente la dejan sin interactuar con la misma. Esta metrica solo se tiene en cuenta si es la primera página que se visitó del sitio web.
- exitrates: De la cantidad total de visitas a las páginas del sitio web, el porcentaje de usuarios que lo abandonaron en esta página. Esto es, el  porcentaje de usuarios que su última visita al sitio fué en esta página.
- pagevalues: Este es el valor promedio del sitio web, indica la contribución que este sitio web hizo al visitante que llega a la página o sección de compra final. Más detalles sobre pageValues: https://support.google.com/analytics/answer/2695658?hl=en
-  specialday: Es una fecha especial o no (1 o 0)
- operatingsystems: Sistema operativo
- browser: Nombre del navegador
- region: Region geográfica del usuario
- traffictype: Tipo de tráfico web
- visitortype: Nuevo o uno que retorno al sitio
- Weekend: 1 si es fin de semana y 0 en otro caso
- revenue: 1 si el usuario hizo una compra y 0 en otro caso

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('online-shoppers-intention.csv')

Leemos el csv:

In [ ]:
df.sample(5)

¿ Cuántas filas y columnas tiene el dataset ?

In [ ]:
n_filas = df.shape[0]
n_columnas = df.shape[1]
print(f"num de filas: {n_filas}")
print(f"num de columnas: {n_columnas}")

## Valores nulos

¿ Hay valores nulos en el dataset ?

¿ Cuántos por cada columna ?

In [ ]:
df.isna().sum()

En porcentajes:

In [ ]:
round(df.isna().mean() * 100, 2)

Tenemos valores nulos en varias columnas. En algunas el porcentaje es muy alto (80% en operating systems) y en otras muy bajo (1% en region).

A simple vista:

1) ¿Descartarían alguna columna por completo?

2) ¿Descartarían las filas con nulos en alguna de las columnas?

3) ¿En qué casos creen que conviene utilizar imputación univariante y en cuáles multivariante ?

Vamos a analizar el dataset y buscar una respuesta a estas preguntas.

### Columna administrative

Sabemos que la columna **administrative** que tiene 15% de valores nulos nos dice:

*Número de veces que el usuario visito la sección "administrative"*

Pero la columna **administrative_duration** está relacionada a la misma:

*Tiempo que el usuario paso en la sección administrative*

tiene 0 nulos.

En este caso, al comprender el problema, podemos suponer que cuando  administrative duration tiene un valor > 0, la variable administrative debería ser almenos 1.

Antes de utilizar esta condición para completar nulos en la columna, vamos a verificar que se cumpla la condición.

¿ Existen casos en que Administrative_Duration sea mayor a 0 y Administrative igual a 0 ?


In [ ]:
df[(df.Administrative_Duration > 0) & (df.Administrative == 0)].shape[0]

Vemos que no hay casos, por lo tanto podemos asumir que cuando Administrative_Duration es > 0 entonces Administrative es 1.

Ahora..

¿ Qué pasa si Administrative_Duration es = 0 ?

¿ Hay casos en que Administrative_Duration sea = 0 y Administrative = 1 ?

In [ ]:
df[(df.Administrative_Duration == 0) & (df.Administrative == 1)].shape[0]

Tenemos 78 casos. No parecen ser muchos.

Ahora, ¿ cuántas veces sucede lo contrario ? (administrative duration == 0 y administrative == 0)

In [ ]:
df[(df.Administrative_Duration == 0) & (df.Administrative == 0)].shape[0]

En la mayoría de los casos, si administrative_duration es = 0, administrative también lo es.

Después de haber analizado el problema, decidimos:

- Cuando administrative duration sea 0: Vamos a completar los nulos de Administrative con 0.

- Cuando administrative duration sea > 0: Vamos a completar los nulos de administrative con 1.

¿ Cómo hacemos esto en python ?

In [ ]:
# Mask
condition_1 = df.Administrative_Duration == 0
condition_2 = df.Administrative_Duration > 0

df.loc[condition_1, 'Administrative'].head()

In [ ]:
df.loc[condition_2, 'Administrative'].head()

Utilizamos las mascaras combinadas con el método .fillna() de pandas.

In [ ]:
df.loc[condition_1, 'Administrative'] = df.loc[condition_1, 'Administrative'].fillna(0) # Cuando la duracion es 0
df.loc[condition_2, 'Administrative'] = df.loc[condition_2, 'Administrative'].fillna(1) # Cuando la duracion es > 0

In [ ]:
df.Administrative.isna().sum()

Ahora la columna administrative no tiene más nulos. Los imputamos teniendo en cuenta el valor de otra columna (y entendiendo el problema)

In [ ]:
df.mean()*100

### Informational duration

En esta columna tenemos un 30% de valores nulos.

En la columna informational, no hay nulos. Esta columna nos puede llegar a servir para encontrar alguna condición como en el caso anterior. Sin embargo, a diferencia del caso anterior donde teniamos una variable "booleana", la columna informational duration es continua.

En estos casos podemos utilizar estrategias como:
- Completar con el valor más frecuente en la columna
- Completar con la media de la columna
- Completar con la mediana de la columna
- Completar con un valor definido por nosotros

Para esto vamos a utilizar el SimpleImputer de scikit learn. Funciona de una manera muy similar a .fillna() de pandas.

Documentación: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer

Algo que debemos tener en cuenta, es que cuando la variable informational es = 0, la variable informational duration debería tambien ser 0. (similar al caso anterior).

Podemos completar los nulos de la misma forma que el caso anterior cuando se cumpla esta condición. En el resto de los casos (cuando informational sea 1), utilizaremos el SimpleImputer de scikitlearn.

In [ ]:
# Mask
informational_0 = df.Informational == 0
informational_1 = df.Informational == 1

In [ ]:
df.loc[informational_0, 'Informational_Duration'] = df.loc[informational_0, 'Informational_Duration'].fillna(0) # Cuando la columna informational es 0, le asignamos una duración 0

In [ ]:
df.Informational_Duration.isna().sum()

Quedan solo 551 valores nulos en la columna.

Ahora, ¿Cómo utilizamos el SimpleImputer en el resto de los casos?

Si vemos la documentación, la clase SimpleImputer tiene los parámetros:

- missing_values
- strategy
- fill_value
- verbose
- copy
- add_indicator

Antes de arrancar a utilizarlo, leamos la documentación y entendamos para que sirve cada uno de estos parámetros, que tipo de datos podemos pasarle y distingamos cuáles son obligatorios y cuáles no.


Una vez que entendimos esto, creamos nuestro objeto SimpleImputer. Para esto primero que nada, debemos importarlo:

In [ ]:
from sklearn.impute import SimpleImputer

Como vimos en sklearn se suelen seguir 3 pasos: crear el objeto, fit y transform.

Paso 1: Creamos el objeto.

Un parametro que tenemos que pasar, es la estrategia. Debemos decidir cual utilizar. Veamos la distribución de la variable en los casos en que informational es = 1

In [ ]:
fig = plt.figure(figsize=(8,4))
sns.distplot(df[informational_1].Informational_Duration)
plt.title("Distribución de la variable informational duration cuando Informational = 1")
plt.show()

In [ ]:
df[informational_1].Informational_Duration.mean()

In [ ]:
df[informational_1].Informational_Duration.median()

Cuando tenemos una distribución "normal" utilizar la media o mediana para completar nulos no nos va a cambiar mucho.

En distribuciones con una cola larga como esta que vemos, suele ser mejor utilizar la mediana ya que la media esta sesgada por los valores extremos.

Por lo tanto, vamos a utilizar la mediana.

In [ ]:
my_simple_imputer = SimpleImputer(missing_values=np.nan, strategy='median') # Este imputer considera nulos a los np.nan y utiliza la mediana para rellenar

Paso 2:


In [ ]:
my_simple_imputer.fit(df[['Informational_Duration']])

Paso 3:


In [ ]:
df['Informational_Duration'] = my_simple_imputer.transform(df[['Informational_Duration']])

In [ ]:
df.Informational_Duration.isna().sum()

Ahora ya completamos todos los nulos en esta columna también, pero quedan varias más.


# Ejercicio

1) Investigar fit_transform e implementarlo en el caso anterior (Informational_Duration).

2) Con lo que vimos hasta ahora, analizar las variables y decidir estrategias para completar nulos / descartar filas o columnas en cada caso.

Al finalizar, no debe haber ninguna fila con valores nulos.

In [ ]:
df.isna().sum()